In [1]:
%pip install scikit-learn


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import numpy as np
import sys
sys.path.append('../')
from utils import remove_similar_rows_per_player, find_lines_with_player, del_patterns, map_emoji_to_description
import emoji
from googletrans import Translator



In [3]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)

In [4]:
# Filter out the German data and reindex
df_ger = df[df["language"] == "de"]
df_ger = df_ger.reset_index(drop=True)

In [5]:
# Remove the similiar rows (The Function is imported from utils on top)
df = remove_similar_rows_per_player(df_ger, df_ger['player'].unique())


In [6]:
# transform data into lower case
data_lower = df.copy()

data_lower['data'] = data_lower['data'].str.lower()
data_lower['player'] = data_lower['player'].str.lower()
data_lower.head()

data_wo_pattern = data_lower.copy()

# delete content patterns
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))

In [7]:
pattern = ['nutze kicker', 
           'mit dem', 
           'nur €2,49', 
           'bereits pur-abonnent?', 
           'alle antworten', 
           'hinweis zur verarbeitung', 
           'werbung','olympia-verlags', 
           'bild', 'g+j medien gmbh', 
           'services',
           'kurz-link dieses artikels lautet:', 
           'http://epaper.welt.de',
           'stephan von nocks',
           'www.faz.net',
           'mcfit mitgliedschaft',
           'fitx-vertrag',
           'kündigeneasyfitness',
           'proteinbedarf',
           'fitnessland',
           'kündigeneasyfitness',
           'trainingspuls berechnen',
           'pulsrechner',
           'fitseveneleven-mitgliedschaft',
           'alkoholabbau & promille',
           'index.promillerechner',
           'mitgliedschaft kündigen',
           'promillerechner',
           'ihr body-mass-index',
           'bmi rechner',
           'kalorienrechner',
           'grundumsatz & kalorienbedarf',
           'partnerangebote',
           'newsletter',
           'journalismus der presse',
           'abonnieren',
           '(apa)',
           'foto:',
           'quelle:',
           'lesezeit:',
           'lesen sie mehr',
           'stellenmarkt der sz.',
           'bewerben sie sich jetzt',
           'gutscheine',
           'vergleichsportal',
           'stern plus-inhalten',
           'jederzeit kündbar',
           'bereits registriert?',
           'zur startseite',
           'öffnet in neuem tab oder fenster'
           'vollansicht der tabelle unter'
        
           ]


In [8]:
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: del_patterns(str(x), pattern))                                                      

In [9]:
data_wo_pattern['data'].iloc[191]

'leverkusens edmond tapsoba, robert andrich, odilon kossounou und kerem demirbay nach dem schlusspfiff.bayer leverkusen und der vfl wolfsburg trennen sich in einem schwachen spiel torlos. der punkt hilft im kampf um die europacup-plätze am ehesten noch der werkself.die siegesserie von bayer leverkusen ist gerissen. für das team von trainer xabi alonso reichte es im auswärtsspiel am sonntag beim vfl wolfsburg nur zu einem 0:0, zuvor hatte bayer fünfmal in folge in der fußball-bundesliga gewonnen. das torlose remis taugt auch nur bedingt als mutmacher für das viertelfinal-rückspiel in der europa league am kommenden donnerstag bei union saint-gilloise (hinspiel 1:1). in der bundesliga-tabelle bleibt leverkusen sechster, der vfl belegt mit vier punkten rückstand weiter den neunten\\xa0platz.bayer-coach alonso rotierte im vergleich zur europacup-partie gegen union saint-gilloise auf fünf positionen und ließ unter anderem die nationalspieler florian wirtz und jonathan tah zunächst auf der ba

In [10]:
data_lower['data'] = data_lower['data'].apply(lambda x: re.sub(r'[^\w\s]', lambda match: map_emoji_to_description(match.group(), language = 'de',), str(x)))

In [11]:
data_lower['data'].iloc[19]

"{'content': 'bayern münchen hat die tabellenführung in der deutschen bundesliga an borussia dortmund verloren. der rekordmeister verlor am sonntag durch zwei kuriose elfmeter nach videobeweis bei bayer leverkusen mit 1:2. das team von trainer julian nagelsmann hat nach der dritten saisonniederlage einen punkt rückstand auf dortmund, das am samstag gegen den 1. fc köln mit 6:1 gewonnen hatte. damit gehen die bayern als verfolger in das duell mit der borussia am 1. april.\\njoshua kimmich (22.) brachte die münchner in führung. der argentinische weltmeister exequiel palacios drehte das spiel mit zwei verwandelten foulelfmetern (56., 73.). schiedsrichter tobias stieler hatte dem gefoulten amine adli wegen vermeintlicher schwalben jeweils zunächst die gelbe karte gegeben. beide male nahm er die karte nach videostudium zurück. leverkusen ist nun dicht an den europacup-plätzen dran.\\ntabellendritter ist weiter union berlin nach einem 2:0-heimsieg gegen eintracht frankfurt, an rb leipzig vor

In [12]:
data_lower['data'].iloc[19]

"{'content': 'bayern münchen hat die tabellenführung in der deutschen bundesliga an borussia dortmund verloren. der rekordmeister verlor am sonntag durch zwei kuriose elfmeter nach videobeweis bei bayer leverkusen mit 1:2. das team von trainer julian nagelsmann hat nach der dritten saisonniederlage einen punkt rückstand auf dortmund, das am samstag gegen den 1. fc köln mit 6:1 gewonnen hatte. damit gehen die bayern als verfolger in das duell mit der borussia am 1. april.\\njoshua kimmich (22.) brachte die münchner in führung. der argentinische weltmeister exequiel palacios drehte das spiel mit zwei verwandelten foulelfmetern (56., 73.). schiedsrichter tobias stieler hatte dem gefoulten amine adli wegen vermeintlicher schwalben jeweils zunächst die gelbe karte gegeben. beide male nahm er die karte nach videostudium zurück. leverkusen ist nun dicht an den europacup-plätzen dran.\\ntabellendritter ist weiter union berlin nach einem 2:0-heimsieg gegen eintracht frankfurt, an rb leipzig vor

## Find sentences with Playername

In [13]:
data_with_playernames = find_lines_with_player(df, df['player'].unique())

In [14]:
data_with_playernames

,data,player,language,publishedAt
0,{'content': 'Trainer Xabi Alonso vor den mitge...,Exequiel Palacios,de,2023-02-24T09:33:31Z
1,{'content': 'Nutze kicker auf seinen digitalen...,Exequiel Palacios,de,2023-03-03T21:35:13Z
2,{'content': 'Nutze kicker auf seinen digitalen...,Exequiel Palacios,de,2023-03-07T11:34:39Z
3,{'content': 'Mitchel Bakker am Mittwoch im Abs...,Exequiel Palacios,de,2023-03-08T14:25:18Z
4,{'content': '© G+J Medien GmbH,Exequiel Palacios,de,2023-03-09T19:36:53Z
...,...,...,...,...
21,"{'content': ""Der SC Freiburg und Bayer Leverku...",Piero Hincapié,de,2023-02-26T16:28:57Z
22,{'content': 'Ein Monaco-Betreuer packt Sardar ...,Piero Hincapié,de,2023-02-22T16:59:33Z
23,{'content': 'Leverkusens Torwart Lukas Hradeck...,Piero Hincapié,de,2023-02-17T17:14:46Z
24,{'content': 'Das WichtigsteTut es in der Kolle...,Piero Hincapié,de,2023-02-17T08:56:42Z


In [15]:
data_with_playernames = find_lines_with_player(data_wo_pattern, data_wo_pattern['player'].unique(), n_lines=5)

In [21]:
data_with_playernames['data'].iloc[19]

"bayern münchen hat die tabellenführung in der deutschen bundesliga an borussia dortmund verloren. der rekordmeister verlor am sonntag durch zwei kuriose elfmeter nach videobeweis bei bayer leverkusen mit 1:2. das team von trainer julian nagelsmann hat nach der dritten saisonniederlage einen punkt rückstand auf dortmund, das am samstag gegen den 1. fc köln mit 6:1 gewonnen hatte. damit gehen die bayern als verfolger in das duell mit der borussia am 1. april.joshua kimmich (22.) brachte die münchner in führung. der argentinische weltmeister exequiel palacios drehte das spiel mit zwei verwandelten foulelfmetern (56., 73.). schiedsrichter tobias stieler hatte dem gefoulten amine adli wegen vermeintlicher schwalben jeweils zunächst die gelbe karte gegeben. beide male nahm er die karte nach videostudium zurück. leverkusen ist nun dicht an den europacup-plätzen dran.tabellendritter ist weiter union berlin nach einem 2:0-heimsieg gegen eintracht frankfurt, an rb leipzig vorbei auf rang vier s

In [22]:
data_with_playernames['data'] = data_with_playernames['data'].apply(lambda x: re.sub(r'[^\w\s]', lambda match: map_emoji_to_description(match.group(), language = 'de',), str(x)))

In [17]:
data_rm = data_with_playernames.copy()

# strip_numeric -> removing digits (https://gensimr.news-r.org/reference/strip_numeric.html)
data_rm['data'] = data_rm['data'].apply(strip_numeric)

# strip_punctutation -> removing punctations (https://gensimr.news-r.org/reference/strip_punctuation.html)
data_rm['data'] = data_rm['data'].apply(strip_punctuation)

# strip multiple whitespaces also \n -> (https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.strip_multiple_whitespaces)
data_rm['data'] = data_rm['data'].apply(strip_multiple_whitespaces)

In [18]:
data_rm['data'].iloc[13]

'der fc bayern ist nicht mehr tabellenführer der fußball bundesliga in leverkusen liegt schiedsrichter tobias stieler zweimal komplett daneben wird zweimal vom videoassistenten korrigiert und zweimal jubelt bayer kurz darauf für die münchner ist es die dritte pleite der saison oliver kahn warf sich verzweifelt in seinem vip sitz hin und her trainer julian nagelsmann schlug sich wütend mit der handfläche gegen die stirn bayern münchen hat im titelrennen nach dem offensivfeuerwerk von borussia dortmund auf kuriose weise gepatzt und ist krachend von der tabellenspitze gestürzt der abonnementmeister verlor bei bayer leverkusen gab eine führung aus der hand und liegt in der tabelle vor dem ligagipfel in zwei wochen plötzlich einen punkt hinter dem bvb dortmund der einzig verbliebene herausforderer um die meisterschaft hatte am gestrigen samstag ein gegen den fc köln vorgelegt der fc bayern ist erstmals seit dem zwölften spieltag nicht tabellenführer leverkusen hat das sehr gut gemacht sie w

## Stopwords

In [19]:
german_stop_words = stopwords.words('german')

# Create a list of stop words to remove 
stop_words_to_remove = ['nicht', 'nichts']

stop_words_to_add = ['vor', 'in', 'den', 'dem', 'beim', 'wir', 'der', 'ist','ende', 'seite', 'ersten', 'fürs', 'eh', 'blick', 'schon', 'zumal', 'erst', 'mal', 'bild', 't', '(dpa)']

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  german_stop_words.remove(word)

german_stop_words.append(stop_words_to_add)

# Define a function to apply remove_stopwords on a column
def remove_stopwords_from_text(text):
    return remove_stopwords(text, stopwords=german_stop_words)

# Apply the remove_stopwords function to the 'text' column using the apply method
data_rm['data'] = data_rm['data'].apply(remove_stopwords_from_text)


## Pattern Regex


In [20]:
'''patternlist = [
    r'^\s*nutze kicker.*?\.',
    r'^\s*die zustimmung.*?\.',
]

for pattern in patternlist:

    data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(pattern, "", str(x), flags=re.M))
'''


'patternlist = [\n    r\'^\\s*nutze kicker.*?\\.\',\n    r\'^\\s*die zustimmung.*?\\.\',\n]\n\nfor pattern in patternlist:\n\n    data_wo_pattern[\'data\'] = data_wo_pattern[\'data\'].apply(lambda x: re.sub(pattern, "", str(x), flags=re.M))\n'